In [1]:
import pandas as pd
import os
import shutil
import numpy as np
from tqdm import tqdm
import ants
import nibabel as nib

In [14]:
df = pd.read_csv('../Data/DS2072.csv')
df['is_nii_gz'] = [file.endswith('.nii.gz') for file in df['local_paths'].values]
df = df[df['is_nii_gz']]

In [4]:
def safe_mkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)
    else:
        pass

In [5]:
def write_json(data,filepath):
    import json
    with open(filepath, 'w') as outfile:
        json.dump(data, outfile)

In [15]:
study_name = 'Intrinsic Brain Architecture of Young Children with Autism While Awake and Asleep'
study_df = df[df['collection_title']==study_name]

In [16]:
study_subjects = np.unique(study_df['subjectkey'].values)
nsubjects = len(study_subjects)

In [30]:
ndar_root = '/mmfs1/data/pijarj/ndar_fmri/' 
bids_root = '/mmfs1/data/pijarj/'

In [24]:
def check_has_anat_and_epi(sub):
    sub_df = study_df.iloc[study_df['subjectkey'].values==sub]
    fmri_idx = sub_df['scan_type'].values=='fMRI'
    anat_idx = sub_df['scan_type'].values=='MR structural (MPRAGE)'
    return fmri_idx.sum()>0 and anat_idx.sum()>0

In [25]:
has_anat_and_epi = np.array([check_has_anat_and_epi(s) for s in study_subjects])

In [26]:
use_subjects = study_subjects[has_anat_and_epi]
nsubjects = len(use_subjects)
print(nsubjects)

26


In [28]:
s = 0
sub = use_subjects[s]
sub_df = study_df.iloc[study_df['subjectkey'].values==sub]
fmri_idx = sub_df['scan_type'].values=='fMRI'
anat_idx = sub_df['scan_type'].values=='MR structural (MPRAGE)'

epi_fn = sub_df.iloc[fmri_idx]['local_paths'].values[0]
anat_fn = sub_df.iloc[anat_idx]['local_paths'].values[0]

epi_path = os.path.join(ndar_root,epi_fn[2::])
anat_path = os.path.join(ndar_root,anat_fn[2::])

epi_dest = os.path.join(bids_root,f'ds-{collection_id}',f'sub-{s+1:03d}','func',f'sub-{s+1:03d}_task-rest_bold.nii.gz')
anat_dest = os.path.join(bids_root,f'ds-{collection_id}',f'sub-{s+1:03d}','anat',f'sub-{s+1:03d}_T1w.nii.gz')

In [31]:
safe_mkdir(os.path.join(bids_root,f'ds-2072'))
for s in range(1,nsubjects+1):
    safe_mkdir(os.path.join(bids_root,f'ds-{collection_id}',f'sub-{s:03d}'))
    safe_mkdir(os.path.join(bids_root,f'ds-{collection_id}',f'sub-{s:03d}','func'))
    safe_mkdir(os.path.join(bids_root,f'ds-{collection_id}',f'sub-{s:03d}','anat'))

In [32]:
for s in tqdm(range(0,nsubjects)):
    sub = use_subjects[s]
    sub_df = study_df.iloc[study_df['subjectkey'].values==sub]
    fmri_idx = sub_df['scan_type'].values=='fMRI'
    anat_idx = sub_df['scan_type'].values=='MR structural (MPRAGE)'

    epi_fn = sub_df.iloc[fmri_idx]['local_paths'].values[0]
    anat_fn = sub_df.iloc[anat_idx]['local_paths'].values[0]

    epi_path = os.path.join(ndar_root,epi_fn[2::])
    anat_path = os.path.join(ndar_root,anat_fn[2::])

    epi_dest = os.path.join(bids_root,f'ds-2072',f'sub-{s+1:03d}','func',f'sub-{s+1:03d}_task-rest_bold.nii.gz')
    anat_dest = os.path.join(bids_root,f'ds-2072',f'sub-{s+1:03d}','anat',f'sub-{s+1:03d}_T1w.nii.gz')

    t1 = ants.image_read(anat_path) 
    bold = ants.image_read(epi_path)

    t1.to_filename(anat_dest)
    bold.to_filename(epi_dest)

    bold_json = {"RepetitionTime" : bold.spacing[-1],
                "TaskName" : 'rest'}
    write_json(bold_json,epi_dest.replace('.nii.gz','.json')) 
    im = nib.load(epi_dest)
    header = im.header.copy()
    header.set_xyzt_units(xyz='mm', t='sec')
    nib.nifti1.Nifti1Image(im.get_fdata(), None, header=header).to_filename(epi_dest)
    assert nib.load(epi_dest).header.get_xyzt_units()==('mm', 'sec'),'timing missing from header'

100%|██████████| 26/26 [06:49<00:00, 15.74s/it]


In [1]:
import json
data = {
    "Name" : study_df["collection_title"].values[0] ,
    "RepetitionTime": 2.0,
    "SliceTiming" : 2.0 ,
    "TaskName" : "taskrest" ,
    "BIDSVersion" : "20.2.0"}
json_string = json.dumps(data)
print(json_string)
with open(os.path.join(bids_root,f'ds-2072','dataset_description.json'), 'w') as outfile:
    json.dump(json_string, outfile)

NameError: name 'study_df' is not defined